In [1]:
import sys
sys.path.insert(0, '../' )

import time


In [2]:
from libgf264fft import clib_wrapper as cgf

# defining gf
from utils import gf2192 as gf

# the commit scheme
from commit import merkeltreecommit as mt

# hash for challenges
from utils import hash as H

from fri_ldt import m_fri_ldt as fri

def _dummy( *params ) : return None


In [3]:
def _log2( w ):
    assert 0 != w
    assert 0== (w&(w-1))
    r, t = 0 , 1
    while(w!=t): r , t = r+1 , t<<1
    return r

def _pad_len( n_or_m ):
    r = 1
    while( r < n_or_m ):  r = r<<1
    return r

In [4]:
from utils import randombytes as rd

#rd.randombytes(16)


from aesR1CS import aes128

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct, rd_states , rk , rk_tmp = aes128.aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( rd_states , rk , rk_tmp  )
print( "ct0", ct )



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
b'_rd\x15W\xf5\xbc\x92\xf7\xbe;)\x1d\xb9\xf9\x1a\xff\x87\x96\x841\xd8jQdQQ\xfaw:\xd0\tL\x9c\x1ef\xf7q\xf0v,?\x86\x8eSM\xf2Vc\x85\xb7\x9f\xfcS\x8d\xf9\x97\xbeG\x8euG\xd6\x91\xf4\xbc\xd4T2\xe5T\xd0u\xf1\xd6\xc5\x1d\xd0;<\x98\x16\xeet\x00\xf8\x7fUk,\x04\x9c\x8eZ\xd06\xc5~\x1c\x15\x9a\x9b\xd2\x86\xf0_K\xe0\x98\xc649\xba\xa0=\xe7\xa1\xf9\xb5n\xd5Q,\xba_AM#\xe9\xf7N\xec\x020 \xf6\x1b\xf2\xcc\xf25<!\xc7' b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\xd6\xaat\xfd\xd2\xafr\xfa\xda\xa6x\xf1\xd6\xabv\xfe\xb6\x92\xcf\x0bd=\xbd\xf1\xbe\x9b\xc5\x00h0\xb3\xfe\xb6\xfftN\xd2\xc2\xc9\xbflY\x0c\xbf\x04i\xbfAG\xf7\xf7\xbc\x955>\x03\xf9l2\xbc\xfd\x05\x8d\xfd<\xaa\xa3\xe8\xa9\x9f\x9d\xebP\xf3\xafW\xad\xf6"\xaa^9\x0f}\xf7\xa6\x92\x96\xa7U=\xc1\n\xa3\x1fk\x14\xf9p\x1a\xe3_\xe2\x8cD\n\xdfMN\xa9\xc0&GC\x875\xa4\x1ce\xb9\xe0\x16\xba\xf4\xae\xbfz\xd2T\x992\xd1\xf0\x85Wh\x10\x93\xed\x9c\xb

In [5]:
from aesR1CS import aes128R1CS_z as R1CSz

import importlib
importlib.reload( R1CSz )

z = R1CSz.get_vec_z(pt,sk,64)

print( R1CSz.aes128R1CS_num_constrains , len(z) )


3656 3320


In [6]:
from aesR1CS import r1cs

MATs = r1cs.get_r1cs( R1CSz.aes128R1CS_num_constrains , len(z) , 64, 128)  # AES128

mat_a = MATs['A']
mat_b = MATs['B']
mat_c = MATs['C']


In [7]:
def mat_x_vec( mat , vec , n_row=-1) :
    if 0 > n_row : n_row = len(mat)
    r = [0]*n_row
    for i in range(n_row):
        ri = 0
        for j in range(len(vec)):
            ri ^= cgf.gf264_mul( mat[i][j] , vec[j] )
        r[i] = ri
    return r


In [8]:
print( "calculate Az, Bz, Cz" )
st = time.time()
Az = mat_x_vec( mat_a , z )
Bz = mat_x_vec( mat_b , z )
Cz = mat_x_vec( mat_c , z )
ed = time.time()
print( "time:", format(ed-st) , "secs" )




calculate Az, Bz, Cz
time: 24.987906455993652 secs


In [9]:


def process_R1CS( R1CS , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    mat_A , mat_B , mat_C , vec_z , witness_idx = R1CS
    n = len(vec_z)
    m = len(mat_A)
    pad_len = _pad_len( max(n,m) )
    inst_dim = _log2(witness_idx)
    dump( f"pad_len: {pad_len}, inst_dim: {inst_dim}" )

    dump( "padding and calcuate f_1v, f_z, f_w" )
    st = time.time()
    p_vec_z = vec_z + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(vec_z)) ]   # ZK
    #p_vec_z = vec_z + [ 0 for _ in range(pad_len-len(vec_z)) ]    # no ZK
    f_z = gf.ifft( p_vec_z , 1 , 0 );   p_vec_z.extend( gf.fft( f_z , 1 , pad_len ) )
    f_w = gf.polydiv( f_z , inst_dim )[witness_idx:] + [0]*witness_idx
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )
    dump( f"f_z : [{len(f_z)}] ...[{witness_idx-4}:{witness_idx+4}]:", f_z[witness_idx-4:witness_idx+4] , "..." )
    dump( f"f_w : (f_z - f_1v) /Z_{inst_dim} : " , f_w[:4] , "..." )
    #f_1v = cgf.ibtfy( vec_z[:witness_idx] , 0 )   # shouldn't compute it actually. do this for checking correctness only.
    #dump( f"f_1v : [{len(f_1v)}] ...[{witness_idx-4}:]:", f_1v[witness_idx-4:] )

    ## test ##
    #dump( f"f_1v: [{len(f_1v)}]", f_1v )
    #v_1v_256 = cgf.btfy( f_1v , 256 )
    #v_fw     = gf.fft( f_w , 1 , 256 )
    #for i in range( witness_idx ):
    #    cc0 = (p_vec_z[256+i] ^ v_1v_256[i])
    #    cc1 = cgf.gf264_mul( v_fw[i] , cgf.index_to_gf264((256+i)>>inst_dim) )
    #    dump( f"check {i}: ({p_vec_z[256+i]} ^ {v_1v_256[i]}) == Z_{inst_dim} x {v_fw[i]}" , cc0 == cc1 )


    # use global variable
    #dump( "calculate Az, Bz, Cz" )
    #st = time.time()
    #Az = mat_x_vec( mat_A , vec_z )
    #Bz = mat_x_vec( mat_B , vec_z )
    #Cz = mat_x_vec( mat_C , vec_z )
    #ed = time.time()
    #dump( "time:", format(ed-st) , "secs" )
    # XXX: check if Az*Bz == Cz here

    dump( "calculate f_Az, f_Bz, f_Cz" )
    st = time.time()
    v_Az    = Az + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(Az)) ]
    v_Bz    = Bz + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(Bz)) ]
    v_Cz    = Cz + [ gf.mul(v_Az[i],v_Bz[i]) for i in range(m,pad_len) ]           # !!! XXX: need to discuss here
    f_Az = gf.ifft( v_Az , 1 , 0 )
    f_Bz = gf.ifft( v_Bz , 1 , 0 )
    f_Cz = gf.ifft( v_Cz , 1 , 0 )
    v_Az.extend( gf.fft( f_Az , 1 , pad_len ) )
    v_Bz.extend( gf.fft( f_Bz , 1 , pad_len ) )
    v_Cz.extend( gf.fft( f_Bz , 1 , pad_len ) )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )

    return f_w , f_z , p_vec_z , f_Az , f_Bz , f_Cz , v_Az , v_Bz , v_Cz


In [10]:

def row_check( v_Az , v_Bz , v_Cz , pad_len , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    ## row check
    dump( f"row check: calculate (f_Azxf_Bz - f_Cz) / Z_{pad_len}" )
    st = time.time()
    v_AzxBz = v_Cz[:pad_len] + [ gf.mul(v_Az[i],v_Bz[i]) for i in range(pad_len,2*pad_len) ]
    f_Azxf_Bz = gf.ifft( v_AzxBz , 1 , 0 )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )
    dump( f"f_Azxf_Bz: [{len(f_Azxf_Bz)}] ...[{pad_len-2}:{pad_len+2}]" , f_Azxf_Bz[pad_len-2:pad_len+2] , "..." )
    #dump( f"f_Cz     : [{len(f_Cz)}] ...[{pad_len-4}:]" , f_Cz[pad_len-4:] )
    f_rowcheck = f_Azxf_Bz[pad_len:]
    dump( f"f_AB-C / Zh: [{len(f_rowcheck)}] " , f_rowcheck[:2] , "..." )
    return f_rowcheck


In [ ]:
def commit_polys( polys , max_poly_len , RS_rho = 8 , offset = (1<<63) , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    dump( "commit polys: (poly_len):" , [ len(p) for p in polys ] )
    mesg0 = []
    for pp in polys :
        v_p0 = gf.fft( pp , RS_rho * (max_poly_len//len(pp)) , offset )
        v_p1 = [ int.to_bytes(x,gf.GF_BSIZE,'little') for x in v_p0 ]
        v_p2 = [ b''.join( (v_p1[2*i] , v_p1[2*i+1]) ) for i in range(len(v_p1)//2) ]
        mesg0.append( v_p2 )
    dump( f"mesg0: ({max_poly_len}x{RS_rho}/2 = {max_poly_len*RS_rho//2}) " , [len(v) for v in mesg0] )
    mesgs = [ b''.join(e) for e in zip(*mesg0) ]
    dump( f"|mesgs:| = {len(mesgs)} , |mesgs[0]| = {len(mesgs[0])} == {gf.GF_BSIZE * 2 * len(mesg0)}" )
    rt , r_leaf , mktree = mt.commit( mesgs )
    return rt , mesgs , r_leaf , mktree


In [11]:

alpha = gf.from_bytes( H.gen( b'123' , bytes([1,1]) )[:gf.GF_BSIZE] )
print( f"alpha: {hex(alpha)}" )
pad_len = 4096
v_alpha =  [ 1 , alpha ] + [0]*(pad_len-2)
m = len(mat_a)
n = len(mat_a[0])
for i in range(2,m): v_alpha[i] = gf.mul( v_alpha[i-1], v_alpha[1] )
print( "evaluate values of p2A, p2B, p2C" )
st = time.time()
_v_p2A , _v_p2B , _v_p2C = [0]*pad_len , [0]*pad_len , [0]*pad_len
for j in range(n):
        aj, bj, cj = mat_a[0][j] , mat_b[0][j] , mat_c[0][j]
        for i in range(1,m):
            aj ^= gf.mul_gf264( v_alpha[i] , mat_a[i][j] )
            bj ^= gf.mul_gf264( v_alpha[i] , mat_b[i][j] )
            cj ^= gf.mul_gf264( v_alpha[i] , mat_c[i][j] )
        _v_p2A[j] , _v_p2B[j] , _v_p2C[j] = aj , bj , cj
ed = time.time()
print( "time:" , format(ed-st) , "secs" )



alpha: 0x9e04ed9b52db30566646fa76827199aac4e48f4b180c9edc
evaluate values of p2A, p2B, p2C
time: 94.83293843269348 secs


In [12]:

def lincheck_step1( alpha , mat_A , mat_B , mat_C , pad_len , value_or_poly , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    m = len(mat_A)
    n = len(mat_A[0])

    ## lincheck
    dump( "lin-check starts. calculate f_alpha, v_alpha" )
    st = time.time()
    v_alpha =  [ 1 , alpha ] + [0]*(pad_len-2)
    for i in range(2,m): v_alpha[i] = gf.mul( v_alpha[i-1], v_alpha[1] )
    f_alpha = gf.ifft( v_alpha , 1 , 0 )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )

    dump( "lin-check step 1. calculate p2A, p2B, p2C and evaluate their values" )
    dump( "evaluate values of p2A, p2B, p2C" )
    dump( "use global veriables" )
    st = time.time()
    #v_p2A , v_p2B , v_p2C = [0]*pad_len , [0]*pad_len , [0]*pad_len
    #for j in range(n):
    #    aj, bj, cj = mat_A[0][j] , mat_B[0][j] , mat_C[0][j]
    #    for i in range(1,m):
    #        aj ^= gf.mul_gf264( v_alpha[i] , mat_A[i][j] )
    #        bj ^= gf.mul_gf264( v_alpha[i] , mat_B[i][j] )
    #        cj ^= gf.mul_gf264( v_alpha[i] , mat_C[i][j] )
    #    v_p2A[j] , v_p2B[j] , v_p2C[j] = aj , bj , cj
    v_p2A = list(_v_p2A)
    v_p2B = list(_v_p2B)
    v_p2C = list(_v_p2C)
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    dump( "interpolate p2A, p2B, p2C" )
    st = time.time()
    p2A , p2B , p2C = gf.ifft( v_p2A , 1 , 0 ) , gf.ifft( v_p2B , 1 , 0 ) , gf.ifft( v_p2C , 1 , 0 )
    if value_or_poly :
        v_alpha.extend( gf.fft(f_alpha,1,pad_len) )
        v_p2A.extend( gf.fft(p2A,1,pad_len) )
        v_p2B.extend( gf.fft(p2B,1,pad_len) )
        v_p2C.extend( gf.fft(p2C,1,pad_len) )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    dump( "return v_alpha, v_p2A, v_p2B, b_p2C" if value_or_poly else "return f_alpha, p2A , p2B , p2C" )
    return (v_alpha , v_p2A , v_p2B , v_p2C ) if value_or_poly else ( f_alpha, p2A , p2B , p2C )


In [18]:

def lincheck_step2( v_alpha , v_p2A , v_p2B , v_p2C ,  p_vec_z , v_Az , v_Bz , v_Cz ,
              s1 , s2 , s3 , r_lincheck , pad_len , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    dump( f"lin-check step 2. poly muls and /Z_{_log2(pad_len)}" )
    st = time.time()
    v_sA = [ gf.mul(v_Az[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2A[i]) for i in range(2*pad_len) ]
    v_sB = [ gf.mul(v_Bz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2B[i]) for i in range(2*pad_len) ]
    v_sC = [ gf.mul(v_Cz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2C[i]) for i in range(2*pad_len) ]
    f_sA = gf.ifft( v_sA , 1 , 0 )
    f_sB = gf.ifft( v_sB , 1 , 0 )
    f_sC = gf.ifft( v_sC , 1 , 0 )
    g = [ gf.mul(s1,f_sA[i])^gf.mul(s2,f_sB[i])^gf.mul(s3,f_sC[i])^r_lincheck[i] for i in range(pad_len) ]
    h = [ gf.mul(s1,f_sA[i])^gf.mul(s2,f_sB[i])^gf.mul(s3,f_sC[i])^r_lincheck[i] for i in range(pad_len,2*pad_len) ]
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    dump( f"g: [{pad_len}] ...[{pad_len-2}:{pad_len}]", g[pad_len-2:] )
    dump( f"h: [{pad_len}] ...[{pad_len-2}:{pad_len}]", h[pad_len-2:] )

    return g , h

In [24]:

def generate_proof( R1CS , h_state , RS_rho = 8 , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    ## process R1CS
    mat_A , mat_B , mat_C , vec_z , witness_idx = R1CS
    n = len(vec_z)
    m = len(mat_A)
    pad_len = _pad_len( max(n,m) )
    dump( f"m(#rows): {m} x n: {n}, witness_idx: {witness_idx}, pad_len: {pad_len}" )

    f_w , f_z , p_vec_z , f_Az , f_Bz , f_Cz , v_Az , v_Bz , v_Cz = process_R1CS( R1CS , verbose )
    ## row check
    f_rowcheck = row_check(v_Az, v_Bz, v_Cz , pad_len , verbose )
    ## generate random polynomials of degree 2xpad_len: r_lincheck and r_ldt
    v_r_lincheck = [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len*2) ]
    for i in range(pad_len): v_r_lincheck[0] ^= v_r_lincheck[i]
    r_lincheck = gf.ifft(v_r_lincheck, 1 , 0 )
    dump( f"r_lincheck: [{len(r_lincheck)}]: ...[{pad_len-2}:{pad_len+2}] ...", r_lincheck[pad_len-2:pad_len+2] )
    r_ldt = [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len*2) ]

    ## first commit them: HASH their RS codeword
    dump( "commit f_w, f_Az, f_Bz , f_Cz, r_lincheck , r_ldt" )
    offset = 1<<63
    rt0 , mesgs0 , r_leaf0 , mktree0 = commit_polys( [f_w , f_Az , f_Bz , f_Cz , r_lincheck , r_ldt ] , 2*pad_len , RS_rho , offset , verbose )
    #h_state = H.gen( h_state , rt0 )  # XXX: skip this step for a fixed alpha for faster lincheck_step1

    ## lin check
    chals = [ H.gen( h_state , bytes([1,i]) )[:gf.GF_BSIZE] for i in range(1,5) ]
    alpha, s1, s2, s3 = gf.from_bytes(chals[0]), gf.from_bytes(chals[1]), gf.from_bytes(chals[2]), gf.from_bytes(chals[3])
    dump( f"alpha: {hex(alpha)}" )
    vs    = lincheck_step1( alpha , mat_A , mat_B , mat_C , pad_len , 1 , verbose )
    g , h = lincheck_step2( *vs , p_vec_z , v_Az , v_Bz , v_Cz , 0 , 0 , 0  , r_lincheck , pad_len , verbose )
    dump( f"commit h" )
    rt1 , mesgs1 , r_leaf1 , mktree1 = commit_polys( [ h ] , 2*pad_len , RS_rho , offset , verbose )
    h_state = H.gen( *chals )

    ## generate f0 for fri_ldt and perform fri_ldt
    y = [ gf.from_bytes( H.gen( h_state , bytes([2,i]) )[:gf.GF_BSIZE] ) for i in range(1,10) ]
    g_raise = gf.ipolydiv([0]+g[:pad_len-1],0)  # raise g by degree 1. It still has to be raised by pad_len.
    f0 = [ gf.mul(y[0],f_w[i])^gf.mul(y[1],f_Az[i])^gf.mul(y[2],f_Bz[i])^gf.mul(y[3],f_Cz[i])
          ^gf.mul(y[4],f_rowcheck[i])
          ^gf.mul(y[5],r_lincheck[i])^gf.mul(y[6],h[i]) ^r_ldt[i]
          ^gf.mul(y[7],g[i])
          #^gf.mul(y[7],f_z[i])
           for i in range(pad_len) ] + [ 
           gf.mul(y[5],r_lincheck[pad_len+i])^r_ldt[pad_len+i]
          #^gf.mul(y[8],rgi)
           for i,rgi in enumerate(g_raise) ]
    ## LDT f0
    rt2 , mesgs2 , r_leaf2 , mktree2 = commit_polys( [f0] , 2*pad_len , RS_rho , offset , verbose )

    ### test ###
    vv0 = [ gf.from_bytes_x2(mesgs0[0][i*48:i*48+48]) for i in range(6) ]
    v_w0 , v_Az0 , v_Bz0 , v_Cz0 , v_lincheck0 , v_ldt0 = vv0[0],vv0[1],vv0[2],vv0[3],vv0[4],vv0[5]
    v_h0   = gf.from_bytes_x2(mesgs1[0])
    v_f00  = gf.from_bytes_x2(mesgs2[0])
    ### check
    idx = 0
    cc0  = gf.mul(y[0],v_w0[idx])^gf.mul(y[1],v_Az0[idx])^gf.mul(y[2],v_Bz0[idx])^gf.mul(y[3],v_Cz0[idx])
    v_f_rowcheck0 = gf.mul( (gf.mul( v_Az0[idx] , v_Bz0[idx] )^v_Cz0[idx]) , cgf.gf264_inv( cgf.index_to_gf264((offset+idx)>>12) ) )
    cc0 ^= gf.mul(y[4],v_f_rowcheck0)
    cc0 ^= gf.mul(y[5],v_lincheck0[idx])^gf.mul(y[6],v_h0[idx])^v_ldt0[idx]

    f_1v = cgf.ibtfy( vec_z[:witness_idx] , 0 )
    rs_f_1v = cgf.btfy( f_1v , offset )
    v_fz0 = rs_f_1v[idx]^gf.mul_gf264( v_w0[idx] , cgf.index_to_gf264( (offset+idx)>>6 ) )

    f_alpha , p2A , p2B , p2C = lincheck_step1( alpha , mat_A , mat_B , mat_C , pad_len , 0 , verbose )
    rs_f_alpha = gf.fft( f_alpha , 1 , offset )
    rs_f_p2A = gf.fft( p2A , 1 , offset )
    rs_f_p2B = gf.fft( p2B , 1 , offset )
    rs_f_p2C = gf.fft( p2C , 1 , offset )
    v_g0 =  gf.mul( 0 , gf.mul(v_Az0[idx],rs_f_alpha[idx])^gf.mul(rs_f_p2A[idx],v_fz0) ) \
           ^gf.mul( 0 , gf.mul(v_Bz0[idx],rs_f_alpha[idx])^gf.mul(rs_f_p2B[idx],v_fz0) ) \
           ^gf.mul( 0  , gf.mul(v_Cz0[idx],rs_f_alpha[idx])^gf.mul(rs_f_p2C[idx],v_fz0) ) \
           ^ v_lincheck0[idx] ^ gf.mul_gf264( v_h0[idx] , cgf.index_to_gf264((offset+idx)>>12) )
    cc0 ^= gf.mul(y[7],v_g0) 
    #cc0 ^= gf.mul(y[7],v_fz0) 
    #cc0 ^= gf.mul_gf264( gf.mul(y[8],v_g0), cgf.gf264_mul( cgf.index_to_gf264(offset+1) , cgf.index_to_gf264((offset+1)>>12) ) )
    dump( cc0 == v_f00[idx] )

    ## open something here

    return True



##### test ##########

generate_proof( (mat_a,mat_b,mat_c,z,64) , b'123' )


m(#rows): 3656 x n: 3320, witness_idx: 64, pad_len: 4096
pad_len: 4096, inst_dim: 6
padding and calcuate f_1v, f_z, f_w
time: 0.46418166160583496 secs
f_z : [4096] ...[60:68]: [11876712511685630227, 1490032541726768366, 332458469872254918, 200, 18174738647891815068, 6001071716487184560, 13518469962322655972, 233317187178767899] ...
f_w : (f_z - f_1v) /Z_6 :  [312035861849824919481153075885112693817429141698599856202, 1620089944709259369159703407761214468639042161271010349218, 1111820246436568787022056710440764625852962160139292615713, 5034371745644154297404058463775424162739160889560634183514] ...
calculate f_Az, f_Bz, f_Cz
time: 0.08130884170532227 secs
row check: calculate (f_Azxf_Bz - f_Cz) / Z_4096
time: 0.04909801483154297 secs
f_Azxf_Bz: [8192] ...[4094:4098] [1112332786103626910286689101989414588845746918463522363660, 278249813662351961942266302299888621330443320668037989969, 2595075207195707045224833133566647982058709927017697983127, 23146089969476222827368650069832281047184793

True